# CS273A Homework 4 

## Due: Monday November 4th, 2024 (11:59pm)

---
## Instructions


This homework (and subsequent ones) will involve data analysis and reporting on methods and results
using Python code. You will submit a **single PDF file** that contains everything to Gradescope. This includes any text you wish to include to describe your results, the complete code snippets of how you attempted each problem, any figures that were generated, and scans of any work on paper that you wish to include. It is important that you include enough detail that we know how you solved the problem, since otherwise we will be unable to grade it. 

Your homeworks will be given to you as Jupyter notebooks containing the problem descriptions and some template code that will help you get started. You are encouraged to use these starter Jupyter notebooks to complete your assignment and to write your report. This will help you not only ensure that all of the code for the solutions is included, but also will provide an easy way to export your results to a PDF file (for example, doing *print preview* and *printing to pdf*). I recommend liberal use of Markdown cells to create headers for each problem and sub-problem, explaining your implementation/answers, and including any mathematical equations. For parts of the homework you do on paper, scan it in such that it is legible (there are a number of free Android/iOS scanning apps, if you do not have access to a scanner), and include it as an image in the Jupyter notebook.

**Double check that all of your answers are legible on Gradescope, e.g. make sure any text you have written does not get cut off.**

If you have any questions/concerns about using Jupyter notebooks, ask us on EdD. If you decide not to use Jupyter notebooks, but go with Microsoft Word or LaTeX to create your PDF file, make sure that all of the answers can be generated from the code snippets included in the document.
### Summary of Assignment: 100 total points
- Problem 1: A Small Neural Network (30 points)
    - Problem 1.1: Forward Pass (10 points)
    - Problem 1.2: Evaluate Loss (10 points)
    - Problem 1.3: Network Size (10 points)
- Problem 2: Neural Networks on MNIST (35 points)
    - Problem 2.1: Varying the Amount of Training Data (15 points)
    - Problem 2.3: Optimization Curves (10 points)
    - Problem 2.3: Tuning your Neural Network (10 points)
- Problem 3: Convolutional Networks (30 points)
    - Problem 3.1: Model structure (10 points)
    - Problem 3.2: Training (10 points)
    - Problem 3.3: Evaluation (5 points)
    - Problem 3.4: Comparing predictions (5 points)
- Statement of Collaboration (5 points)

Before we get started, let's import some libraries that you will make use of in this assignment. Make sure that you run the code cell below in order to import these libraries.

**Important: In the code block below, we set `seed=1234`. This is to ensure your code has reproducible results and is important for grading. Do not change this. If you are not using the provided Jupyter notebook, make sure to also set the random seed as below.**

**Important: Do not change any codes we give you below, except for those waiting for you to complete. This is to ensure your code has reproducible results and is important for grading.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch

from IPython import display

from sklearn.datasets import fetch_openml            # common data set access
from sklearn.preprocessing import StandardScaler     # scaling transform
from sklearn.model_selection import train_test_split # validation tools
from sklearn.metrics import accuracy_score

from sklearn.neural_network import MLPClassifier     # scikit's MLP

import warnings
warnings.filterwarnings('ignore')

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 1234
np.random.seed(seed)  
torch.manual_seed(seed);

---
## Problem 1: A Small Neural Network


Consider the small neural network given in the image below, which will classify a 3-dimensional feature vector $\mathbf{x}$ into one of three classes ($y = 0, 1, 2$):

![svg image](data:image/svg+xml,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%3F%3E%0A%3Csvg%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%20xmlns%3Axlink%3D%22http%3A//www.w3.org/1999/xlink%22%20width%3D%22297.675pt%22%20height%3D%22195.473pt%22%20viewBox%3D%220%200%20297.675%20195.473%22%20version%3D%221.1%22%3E%0A%3Cdefs%3E%0A%3Cg%3E%0A%3Csymbol%20overflow%3D%22visible%22%20id%3D%22glyph0-0%22%3E%0A%3Cpath%20style%3D%22stroke%3Anone%3B%22%20d%3D%22%22/%3E%0A%3C/symbol%3E%0A%3Csymbol%20overflow%3D%22visible%22%20id%3D%22glyph0-1%22%3E%0A%3Cpath%20style%3D%22stroke%3Anone%3B%22%20d%3D%22M%203.328125%20-3.015625%20C%203.390625%20-3.265625%203.625%20-4.1875%204.3125%20-4.1875%20C%204.359375%20-4.1875%204.609375%20-4.1875%204.8125%20-4.0625%20C%204.53125%20-4%204.34375%20-3.765625%204.34375%20-3.515625%20C%204.34375%20-3.359375%204.453125%20-3.171875%204.71875%20-3.171875%20C%204.9375%20-3.171875%205.25%20-3.34375%205.25%20-3.75%20C%205.25%20-4.265625%204.671875%20-4.40625%204.328125%20-4.40625%20C%203.75%20-4.40625%203.40625%20-3.875%203.28125%20-3.65625%20C%203.03125%20-4.3125%202.5%20-4.40625%202.203125%20-4.40625%20C%201.171875%20-4.40625%200.59375%20-3.125%200.59375%20-2.875%20C%200.59375%20-2.765625%200.703125%20-2.765625%200.71875%20-2.765625%20C%200.796875%20-2.765625%200.828125%20-2.796875%200.84375%20-2.875%20C%201.1875%20-3.9375%201.84375%20-4.1875%202.1875%20-4.1875%20C%202.375%20-4.1875%202.71875%20-4.09375%202.71875%20-3.515625%20C%202.71875%20-3.203125%202.546875%20-2.546875%202.1875%20-1.140625%20C%202.03125%20-0.53125%201.671875%20-0.109375%201.234375%20-0.109375%20C%201.171875%20-0.109375%200.953125%20-0.109375%200.734375%20-0.234375%20C%200.984375%20-0.296875%201.203125%20-0.5%201.203125%20-0.78125%20C%201.203125%20-1.046875%200.984375%20-1.125%200.84375%20-1.125%20C%200.53125%20-1.125%200.296875%20-0.875%200.296875%20-0.546875%20C%200.296875%20-0.09375%200.78125%200.109375%201.21875%200.109375%20C%201.890625%200.109375%202.25%20-0.59375%202.265625%20-0.640625%20C%202.390625%20-0.28125%202.75%200.109375%203.34375%200.109375%20C%204.375%200.109375%204.9375%20-1.171875%204.9375%20-1.421875%20C%204.9375%20-1.53125%204.859375%20-1.53125%204.828125%20-1.53125%20C%204.734375%20-1.53125%204.71875%20-1.484375%204.6875%20-1.421875%20C%204.359375%20-0.34375%203.6875%20-0.109375%203.375%20-0.109375%20C%202.984375%20-0.109375%202.828125%20-0.421875%202.828125%20-0.765625%20C%202.828125%20-0.984375%202.875%20-1.203125%202.984375%20-1.640625%20Z%20M%203.328125%20-3.015625%20%22/%3E%0A%3C/symbol%3E%0A%3Csymbol%20overflow%3D%22visible%22%20id%3D%22glyph0-2%22%3E%0A%3Cpath%20style%3D%22stroke%3Anone%3B%22%20d%3D%22M%202.859375%20-6.8125%20C%202.859375%20-6.8125%202.859375%20-6.921875%202.734375%20-6.921875%20C%202.5%20-6.921875%201.78125%20-6.84375%201.515625%20-6.8125%20C%201.4375%20-6.8125%201.328125%20-6.796875%201.328125%20-6.625%20C%201.328125%20-6.5%201.421875%20-6.5%201.5625%20-6.5%20C%202.046875%20-6.5%202.0625%20-6.4375%202.0625%20-6.328125%20L%202.03125%20-6.125%20L%200.59375%20-0.390625%20C%200.546875%20-0.25%200.546875%20-0.234375%200.546875%20-0.171875%20C%200.546875%200.0625%200.75%200.109375%200.84375%200.109375%20C%201%200.109375%201.15625%20-0.015625%201.203125%20-0.15625%20L%201.390625%20-0.90625%20L%201.609375%20-1.796875%20C%201.671875%20-2.03125%201.734375%20-2.25%201.78125%20-2.46875%20C%201.796875%20-2.53125%201.890625%20-2.859375%201.890625%20-2.921875%20C%201.921875%20-3.015625%202.234375%20-3.5625%202.578125%20-3.84375%20C%202.796875%20-4%203.09375%20-4.1875%203.53125%20-4.1875%20C%203.953125%20-4.1875%204.0625%20-3.84375%204.0625%20-3.484375%20C%204.0625%20-2.953125%203.6875%20-1.859375%203.453125%20-1.25%20C%203.375%20-1.03125%203.3125%20-0.90625%203.3125%20-0.703125%20C%203.3125%20-0.234375%203.671875%200.109375%204.140625%200.109375%20C%205.078125%200.109375%205.4375%20-1.34375%205.4375%20-1.421875%20C%205.4375%20-1.53125%205.359375%20-1.53125%205.328125%20-1.53125%20C%205.21875%20-1.53125%205.21875%20-1.5%205.171875%20-1.34375%20C%205.03125%20-0.8125%204.703125%20-0.109375%204.15625%20-0.109375%20C%203.984375%20-0.109375%203.921875%20-0.203125%203.921875%20-0.4375%20C%203.921875%20-0.6875%204%20-0.921875%204.09375%20-1.140625%20C%204.25%20-1.578125%204.703125%20-2.765625%204.703125%20-3.34375%20C%204.703125%20-3.984375%204.3125%20-4.40625%203.5625%20-4.40625%20C%202.9375%20-4.40625%202.453125%20-4.09375%202.078125%20-3.640625%20Z%20M%202.859375%20-6.8125%20%22/%3E%0A%3C/symbol%3E%0A%3Csymbol%20overflow%3D%22visible%22%20id%3D%22glyph0-3%22%3E%0A%3Cpath%20style%3D%22stroke%3Anone%3B%22%20d%3D%22M%203.65625%20-3.984375%20L%204.515625%20-3.984375%20C%204.71875%20-3.984375%204.8125%20-3.984375%204.8125%20-4.1875%20C%204.8125%20-4.296875%204.71875%20-4.296875%204.546875%20-4.296875%20L%203.71875%20-4.296875%20L%203.921875%20-5.4375%20C%203.96875%20-5.640625%204.109375%20-6.34375%204.171875%20-6.46875%20C%204.25%20-6.65625%204.421875%20-6.8125%204.640625%20-6.8125%20C%204.671875%20-6.8125%204.9375%20-6.8125%205.125%20-6.625%20C%204.6875%20-6.59375%204.578125%20-6.234375%204.578125%20-6.09375%20C%204.578125%20-5.859375%204.765625%20-5.734375%204.953125%20-5.734375%20C%205.21875%20-5.734375%205.5%20-5.96875%205.5%20-6.34375%20C%205.5%20-6.796875%205.046875%20-7.03125%204.640625%20-7.03125%20C%204.296875%20-7.03125%203.671875%20-6.84375%203.375%20-5.859375%20C%203.3125%20-5.65625%203.28125%20-5.546875%203.046875%20-4.296875%20L%202.359375%20-4.296875%20C%202.15625%20-4.296875%202.046875%20-4.296875%202.046875%20-4.109375%20C%202.046875%20-3.984375%202.140625%20-3.984375%202.328125%20-3.984375%20L%202.984375%20-3.984375%20L%202.25%20-0.046875%20C%202.0625%200.921875%201.890625%201.828125%201.375%201.828125%20C%201.328125%201.828125%201.09375%201.828125%200.890625%201.640625%20C%201.359375%201.609375%201.453125%201.25%201.453125%201.109375%20C%201.453125%200.875%201.265625%200.75%201.078125%200.75%20C%200.8125%200.75%200.53125%200.984375%200.53125%201.359375%20C%200.53125%201.796875%200.96875%202.046875%201.375%202.046875%20C%201.921875%202.046875%202.328125%201.453125%202.5%201.078125%20C%202.828125%200.453125%203.046875%20-0.75%203.0625%20-0.828125%20Z%20M%203.65625%20-3.984375%20%22/%3E%0A%3C/symbol%3E%0A%3Csymbol%20overflow%3D%22visible%22%20id%3D%22glyph1-0%22%3E%0A%3Cpath%20style%3D%22stroke%3Anone%3B%22%20d%3D%22%22/%3E%0A%3C/symbol%3E%0A%3Csymbol%20overflow%3D%22visible%22%20id%3D%22glyph1-1%22%3E%0A%3Cpath%20style%3D%22stroke%3Anone%3B%22%20d%3D%22M%202.328125%20-4.4375%20C%202.328125%20-4.625%202.328125%20-4.625%202.125%20-4.625%20C%201.671875%20-4.1875%201.046875%20-4.1875%200.765625%20-4.1875%20L%200.765625%20-3.9375%20C%200.921875%20-3.9375%201.390625%20-3.9375%201.765625%20-4.125%20L%201.765625%20-0.578125%20C%201.765625%20-0.34375%201.765625%20-0.25%201.078125%20-0.25%20L%200.8125%20-0.25%20L%200.8125%200%20C%200.9375%200%201.796875%20-0.03125%202.046875%20-0.03125%20C%202.265625%20-0.03125%203.140625%200%203.296875%200%20L%203.296875%20-0.25%20L%203.03125%20-0.25%20C%202.328125%20-0.25%202.328125%20-0.34375%202.328125%20-0.578125%20Z%20M%202.328125%20-4.4375%20%22/%3E%0A%3C/symbol%3E%0A%3Csymbol%20overflow%3D%22visible%22%20id%3D%22glyph1-2%22%3E%0A%3Cpath%20style%3D%22stroke%3Anone%3B%22%20d%3D%22M%203.515625%20-1.265625%20L%203.28125%20-1.265625%20C%203.265625%20-1.109375%203.1875%20-0.703125%203.09375%20-0.640625%20C%203.046875%20-0.59375%202.515625%20-0.59375%202.40625%20-0.59375%20L%201.125%20-0.59375%20C%201.859375%20-1.234375%202.109375%20-1.4375%202.515625%20-1.765625%20C%203.03125%20-2.171875%203.515625%20-2.609375%203.515625%20-3.265625%20C%203.515625%20-4.109375%202.78125%20-4.625%201.890625%20-4.625%20C%201.03125%20-4.625%200.4375%20-4.015625%200.4375%20-3.375%20C%200.4375%20-3.03125%200.734375%20-2.984375%200.8125%20-2.984375%20C%200.96875%20-2.984375%201.171875%20-3.109375%201.171875%20-3.359375%20C%201.171875%20-3.484375%201.125%20-3.734375%200.765625%20-3.734375%20C%200.984375%20-4.21875%201.453125%20-4.375%201.78125%20-4.375%20C%202.484375%20-4.375%202.84375%20-3.828125%202.84375%20-3.265625%20C%202.84375%20-2.65625%202.40625%20-2.1875%202.1875%20-1.9375%20L%200.515625%20-0.265625%20C%200.4375%20-0.203125%200.4375%20-0.1875%200.4375%200%20L%203.3125%200%20Z%20M%203.515625%20-1.265625%20%22/%3E%0A%3C/symbol%3E%0A%3Csymbol%20overflow%3D%22visible%22%20id%3D%22glyph1-3%22%3E%0A%3Cpath%20style%3D%22stroke%3Anone%3B%22%20d%3D%22M%201.90625%20-2.328125%20C%202.453125%20-2.328125%202.84375%20-1.953125%202.84375%20-1.203125%20C%202.84375%20-0.34375%202.328125%20-0.078125%201.9375%20-0.078125%20C%201.65625%20-0.078125%201.03125%20-0.15625%200.75%20-0.578125%20C%201.078125%20-0.578125%201.15625%20-0.8125%201.15625%20-0.96875%20C%201.15625%20-1.1875%200.984375%20-1.34375%200.765625%20-1.34375%20C%200.578125%20-1.34375%200.375%20-1.21875%200.375%20-0.9375%20C%200.375%20-0.28125%201.09375%200.140625%201.9375%200.140625%20C%202.90625%200.140625%203.578125%20-0.515625%203.578125%20-1.203125%20C%203.578125%20-1.75%203.140625%20-2.296875%202.375%20-2.453125%20C%203.09375%20-2.71875%203.359375%20-3.234375%203.359375%20-3.671875%20C%203.359375%20-4.21875%202.734375%20-4.625%201.953125%20-4.625%20C%201.1875%20-4.625%200.59375%20-4.25%200.59375%20-3.6875%20C%200.59375%20-3.453125%200.75%20-3.328125%200.953125%20-3.328125%20C%201.171875%20-3.328125%201.3125%20-3.484375%201.3125%20-3.671875%20C%201.3125%20-3.875%201.171875%20-4.03125%200.953125%20-4.046875%20C%201.203125%20-4.34375%201.671875%20-4.421875%201.9375%20-4.421875%20C%202.25%20-4.421875%202.6875%20-4.265625%202.6875%20-3.671875%20C%202.6875%20-3.375%202.59375%20-3.046875%202.40625%20-2.84375%20C%202.1875%20-2.578125%201.984375%20-2.5625%201.640625%20-2.53125%20C%201.46875%20-2.515625%201.453125%20-2.515625%201.421875%20-2.515625%20C%201.40625%20-2.515625%201.34375%20-2.5%201.34375%20-2.421875%20C%201.34375%20-2.328125%201.40625%20-2.328125%201.53125%20-2.328125%20Z%20M%201.90625%20-2.328125%20%22/%3E%0A%3C/symbol%3E%0A%3Csymbol%20overflow%3D%22visible%22%20id%3D%22glyph2-0%22%3E%0A%3Cpath%20style%3D%22stroke%3Anone%3B%22%20d%3D%22%22/%3E%0A%3C/symbol%3E%0A%3Csymbol%20overflow%3D%22visible%22%20id%3D%22glyph2-1%22%3E%0A%3Cpath%20style%3D%22stroke%3Anone%3B%22%20d%3D%22M%202.9375%20-6.375%20C%202.9375%20-6.625%202.9375%20-6.640625%202.703125%20-6.640625%20C%202.078125%20-6%201.203125%20-6%200.890625%20-6%20L%200.890625%20-5.6875%20C%201.09375%20-5.6875%201.671875%20-5.6875%202.1875%20-5.953125%20L%202.1875%20-0.78125%20C%202.1875%20-0.421875%202.15625%20-0.3125%201.265625%20-0.3125%20L%200.953125%20-0.3125%20L%200.953125%200%20C%201.296875%20-0.03125%202.15625%20-0.03125%202.5625%20-0.03125%20C%202.953125%20-0.03125%203.828125%20-0.03125%204.171875%200%20L%204.171875%20-0.3125%20L%203.859375%20-0.3125%20C%202.953125%20-0.3125%202.9375%20-0.421875%202.9375%20-0.78125%20Z%20M%202.9375%20-6.375%20%22/%3E%0A%3C/symbol%3E%0A%3C/g%3E%0A%3C/defs%3E%0A%3Cg%20id%3D%22surface1%22%3E%0A%3Cpath%20style%3D%22fill-rule%3Anonzero%3Bfill%3Argb%2875%25%2C90.000916%25%2C75%25%29%3Bfill-opacity%3A1%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C18.000793%25%2C0%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2010.960062%2039.685562%20C%2010.960062%2045.736344%206.053812%2050.642594%20-0.000875%2050.642594%20C%20-6.051656%2050.642594%20-10.957906%2045.736344%20-10.957906%2039.685562%20C%20-10.957906%2033.630875%20-6.051656%2028.724625%20-0.000875%2028.724625%20C%206.053812%2028.724625%2010.960062%2033.630875%2010.960062%2039.685562%20Z%20M%2010.960062%2039.685562%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C19.999695%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph0-1%22%20x%3D%2234.622%22%20y%3D%2277.797%22/%3E%0A%3C/g%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C19.999695%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph1-1%22%20x%3D%2240.316%22%20y%3D%2279.291%22/%3E%0A%3C/g%3E%0A%3Cpath%20style%3D%22fill-rule%3Anonzero%3Bfill%3Argb%2891.999817%25%2C91.999817%25%2C91.999817%25%29%3Bfill-opacity%3A1%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%2818.000793%25%2C18.000793%25%2C18.000793%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2070.335062%2079.369156%20C%2070.335062%2083.7715%2066.76475%2087.341812%2062.362406%2087.341812%20C%2057.960062%2087.341812%2054.393656%2083.7715%2054.393656%2079.369156%20C%2054.393656%2074.970719%2057.960062%2071.400406%2062.362406%2071.400406%20C%2066.76475%2071.400406%2070.335062%2074.970719%2070.335062%2079.369156%20Z%20M%2070.335062%2079.369156%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C0%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph2-1%22%20x%3D%2299.575%22%20y%3D%2239.925%22/%3E%0A%3C/g%3E%0A%3Cpath%20style%3D%22fill-rule%3Anonzero%3Bfill%3Argb%2875%25%2C90.000916%25%2C75%25%29%3Bfill-opacity%3A1%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C18.000793%25%2C0%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2010.960062%20-0.0019375%20C%2010.960062%206.05275%206.053812%2010.959%20-0.000875%2010.959%20C%20-6.051656%2010.959%20-10.957906%206.05275%20-10.957906%20-0.0019375%20C%20-10.957906%20-6.052719%20-6.051656%20-10.958969%20-0.000875%20-10.958969%20C%206.053812%20-10.958969%2010.960062%20-6.052719%2010.960062%20-0.0019375%20Z%20M%2010.960062%20-0.0019375%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C19.999695%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph0-1%22%20x%3D%2234.622%22%20y%3D%22117.482%22/%3E%0A%3C/g%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C19.999695%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph1-2%22%20x%3D%2240.316%22%20y%3D%22118.976%22/%3E%0A%3C/g%3E%0A%3Cpath%20style%3D%22fill-rule%3Anonzero%3Bfill%3Argb%2875%25%2C90.000916%25%2C75%25%29%3Bfill-opacity%3A1%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C18.000793%25%2C0%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2010.960062%20-39.685531%20C%2010.960062%20-33.630844%206.053812%20-28.724594%20-0.000875%20-28.724594%20C%20-6.051656%20-28.724594%20-10.957906%20-33.630844%20-10.957906%20-39.685531%20C%20-10.957906%20-45.736313%20-6.051656%20-50.642563%20-0.000875%20-50.642563%20C%206.053812%20-50.642563%2010.960062%20-45.736313%2010.960062%20-39.685531%20Z%20M%2010.960062%20-39.685531%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C19.999695%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph0-1%22%20x%3D%2234.622%22%20y%3D%22157.167%22/%3E%0A%3C/g%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C19.999695%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph1-3%22%20x%3D%2240.316%22%20y%3D%22158.661%22/%3E%0A%3C/g%3E%0A%3Cpath%20style%3D%22fill-rule%3Anonzero%3Bfill%3Argb%2879.998779%25%2C79.998779%25%2C95.999146%25%29%3Bfill-opacity%3A1%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C23.999023%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20135.686625%2019.841812%20C%20135.686625%2025.8965%20130.776469%2030.80275%20124.725687%2030.80275%20C%20118.674906%2030.80275%20113.768656%2025.8965%20113.768656%2019.841812%20C%20113.768656%2013.791031%20118.674906%208.884781%20124.725687%208.884781%20C%20130.776469%208.884781%20135.686625%2013.791031%20135.686625%2019.841812%20Z%20M%20135.686625%2019.841812%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C0%25%2C19.999695%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph0-2%22%20x%3D%22159.323%22%20y%3D%2298.954%22/%3E%0A%3C/g%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C0%25%2C19.999695%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph1-1%22%20x%3D%22165.063%22%20y%3D%22100.448%22/%3E%0A%3C/g%3E%0A%3Cpath%20style%3D%22fill-rule%3Anonzero%3Bfill%3Argb%2879.998779%25%2C79.998779%25%2C95.999146%25%29%3Bfill-opacity%3A1%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C23.999023%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20135.686625%20-19.841781%20C%20135.686625%20-13.791%20130.776469%20-8.88475%20124.725687%20-8.88475%20C%20118.674906%20-8.88475%20113.768656%20-13.791%20113.768656%20-19.841781%20C%20113.768656%20-25.896469%20118.674906%20-30.802719%20124.725687%20-30.802719%20C%20130.776469%20-30.802719%20135.686625%20-25.896469%20135.686625%20-19.841781%20Z%20M%20135.686625%20-19.841781%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C0%25%2C19.999695%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph0-2%22%20x%3D%22159.323%22%20y%3D%22138.639%22/%3E%0A%3C/g%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C0%25%2C19.999695%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph1-2%22%20x%3D%22165.063%22%20y%3D%22140.133%22/%3E%0A%3C/g%3E%0A%3Cpath%20style%3D%22fill-rule%3Anonzero%3Bfill%3Argb%2891.999817%25%2C91.999817%25%2C91.999817%25%29%3Bfill-opacity%3A1%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%2818.000793%25%2C18.000793%25%2C18.000793%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20179.467875%2079.369156%20C%20179.467875%2083.7715%20175.901469%2087.341812%20171.499125%2087.341812%20C%20167.096781%2087.341812%20163.526469%2083.7715%20163.526469%2079.369156%20C%20163.526469%2074.970719%20167.096781%2071.400406%20171.499125%2071.400406%20C%20175.901469%2071.400406%20179.467875%2074.970719%20179.467875%2079.369156%20Z%20M%20179.467875%2079.369156%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cg%20style%3D%22fill%3Argb%280%25%2C0%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph2-1%22%20x%3D%22208.709%22%20y%3D%2239.925%22/%3E%0A%3C/g%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2011.905375%2037.791031%20L%20107.725687%2022.548844%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20152.03125%2094.269531%20C%20150.710938%2093.796875%20148.613281%2092.675781%20147.222656%2091.535156%20L%20146.613281%2095.375%20C%20148.289062%2094.722656%20150.628906%2094.308594%20152.03125%2094.269531%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2011.905375%201.892594%20L%20107.725687%2017.138687%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20152.03125%2098.214844%20C%20150.628906%2098.175781%20148.289062%2097.757812%20146.613281%2097.109375%20L%20147.222656%20100.945312%20C%20148.613281%2099.804688%20150.710938%2098.6875%20152.03125%2098.214844%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2010.878031%20-34.494125%20L%20109.190531%2012.42775%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20153.101562%20101.648438%20C%20151.742188%20102.007812%20149.382812%20102.273438%20147.589844%20102.125%20L%20149.261719%20105.632812%20C%20150.273438%20104.144531%20151.964844%20102.476562%20153.101562%20101.648438%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2010.878031%2034.494156%20L%20109.190531%20-12.427719%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20153.101562%20130.519531%20C%20151.964844%20129.691406%20150.273438%20128.023438%20149.261719%20126.535156%20L%20147.589844%20130.042969%20C%20149.382812%20129.894531%20151.742188%20130.160156%20153.101562%20130.519531%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2011.905375%20-1.892563%20L%20107.725687%20-17.138656%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20152.03125%20133.953125%20C%20150.710938%20133.480469%20148.613281%20132.363281%20147.222656%20131.222656%20L%20146.613281%20135.058594%20C%20148.289062%20134.410156%20150.628906%20133.992188%20152.03125%20133.953125%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2011.905375%20-37.791%20L%20107.725687%20-22.548813%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20152.03125%20137.898438%20C%20150.628906%20137.859375%20148.289062%20137.445312%20146.613281%20136.792969%20L%20147.222656%20140.632812%20C%20148.613281%20139.492188%20150.710938%20138.371094%20152.03125%20137.898438%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%2859.999084%25%2C59.999084%25%2C59.999084%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2068.921%2073.111344%20L%20112.272562%2031.728531%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%2859.999084%25%2C59.999084%25%2C59.999084%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20155.351562%2087.574219%20C%20154.53125%2086.433594%20153.441406%2084.320312%20152.945312%2082.59375%20L%20150.261719%2085.402344%20C%20152.011719%2085.820312%20154.171875%2086.808594%20155.351562%2087.574219%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%2859.999084%25%2C59.999084%25%2C59.999084%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%2067.186625%2071.697281%20L%20115.565531%20-5.267563%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%2859.999084%25%2C59.999084%25%2C59.999084%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20157.75%20125.300781%20C%20157.234375%20123.992188%20156.695312%20121.675781%20156.636719%20119.878906%20L%20153.347656%20121.945312%20C%20154.941406%20122.78125%20156.796875%20124.265625%20157.75%20125.300781%20%22/%3E%0A%3Cpath%20style%3D%22fill-rule%3Anonzero%3Bfill%3Argb%2895.999146%25%2C79.998779%25%2C79.998779%25%29%3Bfill-opacity%3A1%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%2823.999023%25%2C0%25%2C0%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20229.229594%2039.685562%20C%20229.229594%2045.736344%20224.323344%2050.642594%20218.268656%2050.642594%20C%20212.217875%2050.642594%20207.311625%2045.736344%20207.311625%2039.685562%20C%20207.311625%2033.630875%20212.217875%2028.724625%20218.268656%2028.724625%20C%20224.323344%2028.724625%20229.229594%2033.630875%20229.229594%2039.685562%20Z%20M%20229.229594%2039.685562%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cg%20style%3D%22fill%3Argb%2819.999695%25%2C0%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph0-3%22%20x%3D%22253.298%22%20y%3D%2278.89%22/%3E%0A%3C/g%3E%0A%3Cg%20style%3D%22fill%3Argb%2819.999695%25%2C0%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph1-1%22%20x%3D%22258.175%22%20y%3D%2280.384%22/%3E%0A%3C/g%3E%0A%3Cpath%20style%3D%22fill-rule%3Anonzero%3Bfill%3Argb%2895.999146%25%2C79.998779%25%2C79.998779%25%29%3Bfill-opacity%3A1%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%2823.999023%25%2C0%25%2C0%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20229.229594%20-0.0019375%20C%20229.229594%206.05275%20224.323344%2010.959%20218.268656%2010.959%20C%20212.217875%2010.959%20207.311625%206.05275%20207.311625%20-0.0019375%20C%20207.311625%20-6.052719%20212.217875%20-10.958969%20218.268656%20-10.958969%20C%20224.323344%20-10.958969%20229.229594%20-6.052719%20229.229594%20-0.0019375%20Z%20M%20229.229594%20-0.0019375%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cg%20style%3D%22fill%3Argb%2819.999695%25%2C0%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph0-3%22%20x%3D%22253.298%22%20y%3D%22118.575%22/%3E%0A%3C/g%3E%0A%3Cg%20style%3D%22fill%3Argb%2819.999695%25%2C0%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph1-2%22%20x%3D%22258.175%22%20y%3D%22120.069%22/%3E%0A%3C/g%3E%0A%3Cpath%20style%3D%22fill-rule%3Anonzero%3Bfill%3Argb%2895.999146%25%2C79.998779%25%2C79.998779%25%29%3Bfill-opacity%3A1%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%2823.999023%25%2C0%25%2C0%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20229.229594%20-39.685531%20C%20229.229594%20-33.630844%20224.323344%20-28.724594%20218.268656%20-28.724594%20C%20212.217875%20-28.724594%20207.311625%20-33.630844%20207.311625%20-39.685531%20C%20207.311625%20-45.736313%20212.217875%20-50.642563%20218.268656%20-50.642563%20C%20224.323344%20-50.642563%20229.229594%20-45.736313%20229.229594%20-39.685531%20Z%20M%20229.229594%20-39.685531%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cg%20style%3D%22fill%3Argb%2819.999695%25%2C0%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph0-3%22%20x%3D%22253.298%22%20y%3D%22158.26%22/%3E%0A%3C/g%3E%0A%3Cg%20style%3D%22fill%3Argb%2819.999695%25%2C0%25%2C0%25%29%3Bfill-opacity%3A1%3B%22%3E%0A%20%20%3Cuse%20xlink%3Ahref%3D%22%23glyph1-3%22%20x%3D%22258.175%22%20y%3D%22159.754%22/%3E%0A%3C/g%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20136.518656%2022.345719%20L%20201.428812%2036.111344%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20245.695312%2079.003906%20C%20244.289062%2079.035156%20241.933594%2078.742188%20240.222656%2078.179688%20L%20241.03125%2081.980469%20C%20242.363281%2080.769531%20244.398438%2079.542969%20245.695312%2079.003906%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20134.893656%20-13.369125%20L%20203.745219%2030.443375%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20247.382812%2083.136719%20C%20246.078125%2083.660156%20243.769531%2084.210938%20241.972656%2084.28125%20L%20244.058594%2087.558594%20C%20244.882812%2085.957031%20246.359375%2084.097656%20247.382812%2083.136719%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%2859.999084%25%2C59.999084%25%2C59.999084%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20178.409281%2073.505875%20L%20205.143656%2050.822281%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%2859.999084%25%2C59.999084%25%2C59.999084%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20248.402344%2068.277344%20C%20247.515625%2067.1875%20246.308594%2065.140625%20245.710938%2063.445312%20L%20243.195312%2066.40625%20C%20244.96875%2066.722656%20247.183594%2067.582031%20248.402344%2068.277344%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20136.518656%2017.341812%20L%20201.428812%203.572281%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20245.695312%20113.480469%20C%20244.398438%20112.9375%20242.363281%20111.714844%20241.03125%20110.503906%20L%20240.222656%20114.304688%20C%20241.933594%20113.742188%20244.289062%20113.445312%20245.695312%20113.480469%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20136.518656%20-17.341781%20L%20201.428812%20-3.57225%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20245.695312%20118.6875%20C%20244.289062%20118.722656%20241.933594%20118.425781%20240.222656%20117.863281%20L%20241.03125%20121.664062%20C%20242.363281%20120.453125%20244.398438%20119.230469%20245.695312%20118.6875%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%2859.999084%25%2C59.999084%25%2C59.999084%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20176.100687%2071.560562%20L%20209.530375%2014.830094%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%2859.999084%25%2C59.999084%25%2C59.999084%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20251.601562%20105.269531%20C%20251.125%20103.949219%20250.652344%20101.621094%20250.644531%2099.820312%20L%20247.296875%20101.792969%20C%20248.867188%20102.671875%20250.675781%20104.210938%20251.601562%20105.269531%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20134.893656%2013.369156%20L%20203.745219%20-30.443344%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20247.382812%20149.03125%20C%20246.359375%20148.070312%20244.882812%20146.210938%20244.058594%20144.609375%20L%20241.972656%20147.886719%20C%20243.769531%20147.957031%20246.078125%20148.507812%20247.382812%20149.03125%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20136.518656%20-22.345688%20L%20201.428812%20-36.111313%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%280%25%2C0%25%2C39.99939%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20245.695312%20153.164062%20C%20244.398438%20152.625%20242.363281%20151.398438%20241.03125%20150.1875%20L%20240.222656%20153.988281%20C%20241.933594%20153.425781%20244.289062%20153.132812%20245.695312%20153.164062%20%22/%3E%0A%3Cpath%20style%3D%22fill%3Anone%3Bstroke-width%3A0.79701%3Bstroke-linecap%3Abutt%3Bstroke-linejoin%3Amiter%3Bstroke%3Argb%2859.999084%25%2C59.999084%25%2C59.999084%25%29%3Bstroke-opacity%3A1%3Bstroke-miterlimit%3A10%3B%22%20d%3D%22M%20174.811625%2070.931656%20L%20211.975687%20-23.662094%20%22%20transform%3D%22matrix%281%2C0%2C0%2C-1%2C39.704%2C116.084%29%22/%3E%0A%3Cpath%20style%3D%22%20stroke%3Anone%3Bfill-rule%3Anonzero%3Bfill%3Argb%2859.999084%25%2C59.999084%25%2C59.999084%25%29%3Bfill-opacity%3A1%3B%22%20d%3D%22M%20253.382812%20144.085938%20C%20253.121094%20142.707031%20253.023438%20140.332031%20253.296875%20138.554688%20L%20249.683594%20139.976562%20C%20251.09375%20141.089844%20252.636719%20142.894531%20253.382812%20144.085938%20%22/%3E%0A%3C/g%3E%0A%3C/svg%3E%0A)

You are given an input to this network $\mathbf{x}$, 
$$
\mathbf{x} = \begin{bmatrix} x_1 & x_2 & x_3 \end{bmatrix} = \begin{bmatrix} 1 & 3 & -2 \end{bmatrix}
$$
as well as weights $W$ for the hidden layer and weights $B$ for the output layer. 

$$
W = \begin{bmatrix} 
    w_{01} & w_{11} & w_{21} & w_{31} \\ 
    w_{02} & w_{12} & w_{22} & w_{32} \\
    \end{bmatrix} =
    \begin{bmatrix} 
    1 & -1 & 0 & 5 \\ 
    2 & 1 & 1 & 2 \\
    \end{bmatrix}
$$

$$
B = \begin{bmatrix} 
    \beta_{01} & \beta_{11} & \beta_{21} \\
    \beta_{02} & \beta_{12} & \beta_{22} \\
    \beta_{03} & \beta_{13} & \beta_{23} \\
    \end{bmatrix} =
    \begin{bmatrix}
    4 & -1 & 0 \\
    3 & 0 & 2 \\
    2 & 1 & 1
    \end{bmatrix}
$$

For example, $w_{12}$ is the weight connecting input $x_1$ to hidden node $h_2$; $w_{01}$ is the constant (bias) term for $h_1$, etc.

This network uses the ReLU activation function for the hidden layer, and uses the softmax activation function for the output layer. 

Answer the following questions about this network.

### Problem 1.1 (10 points): Forward Pass
- Given the inputs and weights above, compute the values of the hidden units $h_1, h_2$ and the outputs $f_0, f_1, f_2$. You should do this by hand, i.e. you should not write any code to do the calculation, but feel free to use a calculator to help you do the computations.
- You can optionally use $\LaTeX$ in your answer on the Jupyter notebook. Otherwise, write your answer on paper and include a picture of your answer in this notebook. In order to include an image in Jupyter notebook, save the image in the same directory as the .ipynb file and then write `![caption](image.png)`. Alternatively, you may go to Edit --> Insert Image at the top menu to insert an image into a Markdown cell. **Double check that your image is visible in your PDF submission.**
- What class would the network predict for the input $\mathbf{x}$?

### Problem 1.2 (10 points): Evaluate Loss
Typically when we train neural networks for classification, we seek to minimize the log-loss function. Note that the output of the log-loss function is always nonnegative ($\geq 0$), but can be arbitrarily large (you should pause for a second and make sure you understand why this is true).

- Suppose the true label for the input $\mathbf{x}$ is $y = 1$. What would be the value of our loss function based on the network's prediction for $\mathbf{x}$?
- Suppose instead that the true label for the input $\mathbf{x}$ is $y = 2$. What would be the value of our loss function based on the network's prediction for $\mathbf{x}$?

You are free to use numpy / Python to help you calculate this, but don't use any neural network libraries that will automatically calculate the loss for you.

### Problem 1.3 (10 points): Network Size
- Suppose we change our network so that there are $12$ hidden nodes instead of $2$. How many total parameters (weights and biases) are in our new network?

<div>
    <img src="data:image/svg+xml,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%20standalone%3D%22no%22%3F%3E%0A%3C%21DOCTYPE%20svg%20PUBLIC%20%22-//W3C//DTD%20SVG%201.1//EN%22%20%22http%3A//www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd%22%3E%0A%3Csvg%20stroke-miterlimit%3D%2210%22%20style%3D%22fill-rule%3Anonzero%3Bclip-rule%3Aevenodd%3Bstroke-linecap%3Around%3Bstroke-linejoin%3Around%3B%22%20version%3D%221.1%22%20viewBox%3D%220%200%20288%2072%22%20xml%3Aspace%3D%22preserve%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%20xmlns%3Axlink%3D%22http%3A//www.w3.org/1999/xlink%22%3E%0A%3Cdefs/%3E%0A%3Cg%20id%3D%22Layer-1%22%3E%0A%3Cpath%20d%3D%22M34.042%2035.8741C45.8469%2023.244%2031.1794%2022.6473%2024.2857%2024.1167C17.3921%2025.5861-0.960215%2033.2987%206.07817%2043.4256C13.1166%2053.5525%2023.0237%2056.9377%2052.2446%2053.4091C81.4656%2049.8804%2097.2436%2032.811%20122.962%2029.3111C148.681%2025.8112%20155.118%2039.4093%20155.118%2039.4093%22%20fill%3D%22none%22%20opacity%3D%221%22%20stroke%3D%22%23000000%22%20stroke-linecap%3D%22butt%22%20stroke-linejoin%3D%22round%22%20stroke-width%3D%223%22/%3E%0A%3Cpath%20d%3D%22M254.053%2040.6889C242.328%2053.1071%20256.999%2053.6247%20263.883%2052.1549C270.768%2050.685%20289.071%2043.0512%20281.969%2033.1691C274.868%2023.287%20264.94%2020.0179%20235.741%2023.6051C206.543%2027.1922%20190.872%2043.9744%20165.176%2047.5176C139.48%2051.0607%20132.957%2037.7776%20132.957%2037.7776%22%20fill%3D%22none%22%20opacity%3D%221%22%20stroke%3D%22%23000000%22%20stroke-linecap%3D%22butt%22%20stroke-linejoin%3D%22round%22%20stroke-width%3D%223%22/%3E%0A%3C/g%3E%0A%3C/svg%3E%0A" width="200"/>
</div>

## Problem 2: Neural Networks on MNIST

In this part of the assignment, you will get some hands-on experience working with neural networks. We will be using the scikit-learn implementation of a multi-layer perceptron (MLP). See [here](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) for the corresponding documentation. Although there are specialized Python libraries for neural networks, like [TensorFlow](https://www.tensorflow.org/) and [PyTorch](https://pytorch.org/), in this problem we'll just use scikit-learn since you're already familiar with it.

### Problem 2.0: Setting up the Data
First, we'll load our MNIST dataset and split it into a training set and a testing set. Here you are given code that does this for you, and you only need to run it.

We will use the scikit-learn class `StandardScaler` to standardize both the training and testing features. Notice that we **only** fit the `StandardScaler` on the training data, and *not* the testing data.

In [ ]:
# Load the features and labels for the MNIST dataset
# This might take a minute to download the images.
X, y = fetch_openml('mnist_784', as_frame=False, return_X_y=True)

# Convert labels to integer data type
y = y.astype(int)

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, random_state=seed, shuffle=True)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_tr)
X_tr = scaler.transform(X_tr)     # We can forget about the original values & work
X_te = scaler.transform(X_te)     #  just with the transformed values from here

### Problem 2.1: Varying the amount of training data (15 points)
One reason that neural networks have become popular in recent years is that, for many problems, we now have access to very large datasets. Since neural networks are very flexible models, they are often able to take advantage of these large datasets in order to achieve high levels of accuracy. In this problem, you will vary the amount of training data available to a neural network and see what effect this has on the model's performance.

In this problem, you should use the following settings for your network:
- A single hidden layer with $64$ hidden nodes
- Use the ReLU activation function
- Train the network using stochastic gradient descent (SGD) and a constant learning rate of $0.001$
- Use a batch size of 256
- **Make sure to set `random_state=seed`.**

Your task is to implement the following:
- Train an MLP model (with the above hyperparameter settings) using the first `m_tr` feature vectors in `X_tr`, where `m_tr = [100, 1000, 5000, 10000, 20000, 50000, 63000]`. You should use the `MLPClassifier` class from scikit-learn in your implementation.
- Create a plot of the training error and testing error for your MLP model as a function of the number of training data points.  For comparison, also plot the training and test error rates we found when we trained a logistic regression model on MNIST (these values are provided below). Again, be sure to include an x-label, y-label, and legend in your plot and use a log-scale on the x-axis.
- Give a short (one or two sentences) description of what you see in your plot.   Do you think that more data (beyond these 63000 examples) would continue to improve the model's performance?

**Note** that training a neural network with a lot of data can be **a slow process**. Hence, you should be careful to implement your code such that it runs in a reasonable amount of time. One recommendation is to test your code using only a small subset of the given `m_tr` values, and only run your code with the larger values of `m_tr` once you are certain your code is working. (For reference, it took about 20 minutes to train all models on a quad-core desktop with no GPU.)

In [ ]:
#import time         # helpful if you want to track execution time
#tic = time.time()

train_sizes = [100, 1000, 5000, 10000, 20000, 50000, 63000]

tr_err_mlp = []
te_err_mlp = []
for m_tr in train_sizes:
    ### YOUR CODE STARTS HERE
    

    # print(f'Total elapsed time: {time.time()-tic}')
    
    ### YOUR CODE ENDS HERE

In [ ]:
# When plotting, use these (rounded) values from the similar logistic regression problem solution:
tr_err_lr = np.array([0.   , 0.   , 0.   , 0.   , 0.024, 0.053, 0.057])
te_err_lr = np.array([0.318, 0.149, 0.142, 0.137, 0.119, 0.087, 0.083])

In [ ]:
## YOUR CODE HERE (PLOTTING)

In [ ]:
# DISCUSS

---
### Problem 2.2: Optimization Curves (10 points)

One hyperparameter that can have a significant effect on the optimization of your model, and thus its performance, is the learning rate, which controls the step size in (stochastic) gradient descent. In this problem you will vary the learning rate to see what effect this has on how quickly training converges as well as the effect on the performance of your model.

In this problem, you should use the following settings for your network:
- A single hidden layer with $64$ hidden nodes
- Use the ReLU activation function
- Train the network using stochastic gradient descent (SGD)
- Use a batch size of 256
- Set `n_iter_no_change=100` and `max_iter=100`. This ensures that all of your networks in this problem will train for 100 epochs (an *epoch* is one full pass over the training data).
- Make sure to set `random_state=seed`.

Your task is to:
- Train a neural network with the above settings, but vary the learning rate in `lr = [0.0005, 0.001, 0.005, 0.01]`.
- Create a plot showing the training loss as a function of the training epoch (i.e. the x-axis corresponds to training iterations) for each learning rate above. You should have a single plot with four curves. Make sure to include an x-label, a y-label, and a legend in your plot. (Hint: `MLPClassifier` has an attribute `loss_curve_` that you likely find useful.)
- Include a short description of what you see in your plot.

**Important: To make your code run faster, you should train all of your networks in this problem on only the first 10,000 images of `X_tr`**. In the following cell, you are provided a few lines of code that will create a small training set (with the first 10,000 images in `X_tr`) and a validation set (with the second 10,000 images in `X_tr`). You will use the validation later in Problem 3.3. 

In [ ]:
# Create a smaller training set with the first 10,000 images in X_tr
#   along with a validation set from images 10,000 - 20,000 in X_tr

X_val = X_tr[10000:20000] # Validation set
y_val = y_tr[10000:20000]

X_tr = X_tr[:10000]       # From here on, we will only use these smaller sets,
y_tr = y_tr[:10000]       #  so it's OK to discard the rest of the data

In [ ]:
learning_rates = [0.0005, 0.001, 0.005, 0.01]

err_curves = []

for lr in learning_rates:
    ### YOUR CODE STARTS HERE
    
    
    ### YOUR CODE ENDS HERE

### Problem 2.3: Tuning a Neural Network (10 points)

As you saw in Problem 3.2, there are many hyperparameters of a neural network that can possibly be tuned in order to try to maximize the accuracy of your model. For the final problem of this assignment, it is your job to tune these hyperparameters.

For example, some hyperparameters you might choose to tune are:
- Learning rate
- Depth/width of the hidden layers
- Regularization strength
- Activation functions
- Batch size in stochastic optimization
- etc.

To do this, you should train a network on the training data `X_tr` and evaluate its performance on the validation set `X_val` -- your goal is to achieve the highest possible accuracy on `X_val` by changing the network hyperparameters. **Important: To make your code run faster, you should train all of your networks in this problem on only the first 10,000 images of `X_tr`**. This was already set up for you in Problem 3.2.

Try to find settings that enable you to achieve an error rate smaller than 5% on the validation data. However, tuning neural networks can be difficult; if you cannot achieve this target error rate, be sure to try at least five different neural networks (corresponding to five different settings of the hyperparameters).

In your answer, include a table listing the different hyperparameters that you tried, along with the resulting accuracy on the training and validation sets `X_tr` and `X_val`. Indicate which of these hyperparameter settings you would choose for your final model, and report the accuracy of this final model on the testing set `X_te`.

## Problem 3: Torch and Convolutional Networks

In this problem, we will train a small convolutional neural network and compare it to the "standard" MLP model you built in Problem 2.  Since ``scikit`` does not support CNNs, we will implement a simple CNN model using ``torch``.

The ``torch`` library may take a while to install if it is not yet on your system.  It should be pre-installed on ICS Jupyter Hub (``hub.ics.uci.edu``) and Google CoLab, if you prefer to use those.

### Problem 3.0: Defining the CNN

First, we need to define a CNN model.  This is done for you; it consists of one convolutional layer, a pooling layer to down-sample the hidden nodes, and a standard fully-connected or linear layer.  It contains methods to calculate the 0/1 loss as well as the negative log-likelihood, and trains using the Adam variant of SGD.

It can (optionally) output a real-time plot of the training process at each epoch, if you would like to assess how it is doing.

In [ ]:
import torch 
torch.set_default_dtype(torch.float64)

class myConvNet(object):
    def __init__(self):
        # Initialize parameters: assumes data size! 28x28 and 10 classes
        self.conv_ = torch.nn.Conv2d(1, 16, (5,5), stride=2)  # Be careful when declaring sizes;
        self.pool_ = torch.nn.MaxPool2d(3, stride=2)          # inconsistent sizes will give you
        self.lin_ = torch.nn.Linear(400,10)                   # hard-to-read error messages.
            
    def forward_(self,X):
        """Compute NN forward pass and output class probabilities (as tensor) """
        r1 = self.conv_(X)             # X is (m,1,28,28); R is (m,16,24,24)/2 = (m,16,12,12)
        h1 = torch.relu(r1)            #
        h1_pooled = self.pool_(h1)     # H1 is (m,16,12,12), so H1p is (m,16,10,10)/2 = (m,16,5,5)
        h1_flat = torch.nn.Flatten()(h1_pooled)  # and H1f is (m,400)
        r2 = self.lin_(h1_flat)
        f  = torch.softmax(r2,axis=1)  # Output is (m,10)
        return f 
        
    def parameters(self): 
        return list(self.conv_.parameters())+list(self.pool_.parameters())+list(self.lin_.parameters())
    
    def predict(self,X):
        """Compute NN class predictions (as array) """
        m,n = X.shape
        Xtorch = torch.tensor(X).reshape(m,1,int(np.sqrt(n)),int(np.sqrt(n)))
        return self.classes_[np.argmax(self.forward_(Xtorch).detach().numpy(),axis=1)]   # pick the most probable class

    def J01(self,X,y):   return (y != self.predict(X)).mean()
    def JNLL_(self,X,y): return -torch.log(self.forward_(X)[range(len(y)),y.astype(int)]).mean()

    def fit(self, X,y, batch_size=256, max_iter=100, learning_rate_init=.005, momentum=0.9, alpha=.001, plot=False):
        self.classes_ = np.unique(y)        
        m,n = X.shape
        Xtorch = torch.tensor(X).reshape(m,1,int(np.sqrt(n)),int(np.sqrt(n)))        
        self.loss01, self.lossNLL = [self.J01(X,y)], [float(self.JNLL_(Xtorch,y))]

        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate_init)
        for epoch in range(max_iter):                        # 1 epoch = pass through all data
            pi = np.random.permutation(m)                    # per epoch: permute data order
            for ii,i in enumerate(range(0,m,batch_size)):    # & split into mini-batches
                ivals = pi[i:i+batch_size]
                optimizer.zero_grad()                        # Reset the gradient computation
                Ji = self.JNLL_(Xtorch[ivals,:,:,:],y[ivals])
                Ji.backward()
                optimizer.step()
            self.loss01.append(self.J01(X,y))                # track 0/1 and NLL losses
            self.lossNLL.append(float(self.JNLL_(Xtorch,y)))
            
            if plot:                                         # optionally visualize progress
                display.clear_output(wait=True)
                plt.plot(range(epoch+2),self.loss01,'b-',range(epoch+2),self.lossNLL,'c-')
                plt.title(f'J01: {self.loss01[-1]}, NLL: {self.lossNLL[-1]}')
                plt.draw(); plt.pause(.01);

### Problem 3.1: CNN model structure (10 points)

How many (trainable) parameters are specified in the convolutional network? 
(If you like, you can count them -- you can access them through each trainable element, e.g.,
``myConvNet().conv_._parameters['weights']`` and ``..._parameters['bias']``).  List how many from each layer, and the total.

### Problem 3.2: Training the model (10 points)

Now train your model on ``X_tr``. (Note that this should now include only 10k data points.)  If you like you can plot while training to monitor its progress.  Train for 50 epochs, using a learning rate of .001.

(Note that my simple training process takes these arguments directly into ``fit``, rather than being part of the model properties as is typical in ``scikit``.)

### Problem 3.3: Evaluation and Discussion (5 points)

Evaluate your CNN model's training, validation, and test error.  Compare these to the values you got after optimizing your model's training process in Problem 2.3 (Tuning).  Why do you think these differences occur? (Note that your answer may depend on how well your model in P2.3 did, of course.)

### Problem 3.4: Comparing Predictions (5 points)

Consider the "somewhat ambiguous" data point ``X_val[592]``.  Display the data point (it will look a bit weird since it is already normalized).  Then, use your trained ``MLPClassifier`` model to predict the class probabilities.  If there are other classes with non-negligible probability, are they plausible?  Similarly, find the class probabilities predicted by your CNN model.  Compare the two models' uncertainty.

<div>
    <img src="data:image/svg+xml,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%20standalone%3D%22no%22%3F%3E%0A%3C%21DOCTYPE%20svg%20PUBLIC%20%22-//W3C//DTD%20SVG%201.1//EN%22%20%22http%3A//www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd%22%3E%0A%3Csvg%20stroke-miterlimit%3D%2210%22%20style%3D%22fill-rule%3Anonzero%3Bclip-rule%3Aevenodd%3Bstroke-linecap%3Around%3Bstroke-linejoin%3Around%3B%22%20version%3D%221.1%22%20viewBox%3D%220%200%20288%2072%22%20xml%3Aspace%3D%22preserve%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%20xmlns%3Axlink%3D%22http%3A//www.w3.org/1999/xlink%22%3E%0A%3Cdefs/%3E%0A%3Cg%20id%3D%22Layer-1%22%3E%0A%3Cpath%20d%3D%22M34.042%2035.8741C45.8469%2023.244%2031.1794%2022.6473%2024.2857%2024.1167C17.3921%2025.5861-0.960215%2033.2987%206.07817%2043.4256C13.1166%2053.5525%2023.0237%2056.9377%2052.2446%2053.4091C81.4656%2049.8804%2097.2436%2032.811%20122.962%2029.3111C148.681%2025.8112%20155.118%2039.4093%20155.118%2039.4093%22%20fill%3D%22none%22%20opacity%3D%221%22%20stroke%3D%22%23000000%22%20stroke-linecap%3D%22butt%22%20stroke-linejoin%3D%22round%22%20stroke-width%3D%223%22/%3E%0A%3Cpath%20d%3D%22M254.053%2040.6889C242.328%2053.1071%20256.999%2053.6247%20263.883%2052.1549C270.768%2050.685%20289.071%2043.0512%20281.969%2033.1691C274.868%2023.287%20264.94%2020.0179%20235.741%2023.6051C206.543%2027.1922%20190.872%2043.9744%20165.176%2047.5176C139.48%2051.0607%20132.957%2037.7776%20132.957%2037.7776%22%20fill%3D%22none%22%20opacity%3D%221%22%20stroke%3D%22%23000000%22%20stroke-linecap%3D%22butt%22%20stroke-linejoin%3D%22round%22%20stroke-width%3D%223%22/%3E%0A%3C/g%3E%0A%3C/svg%3E%0A" width="200"/>
</div>

### Statement of Collaboration (5 points)

It is **mandatory** to include a Statement of Collaboration in each submission, with respect to the guidelines below. Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.

All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments, in particular, I encourage the students to organize (perhaps using EdD) to
discuss the task descriptions, requirements, bugs in my code, and the relevant technical content before they start
working on it. However, you should not discuss the specific solutions, and, as a guiding principle, you are not
allowed to take anything written or drawn away from these discussions (i.e. no photographs of the blackboard,
written notes, referring to EdD, etc.). Especially after you have started working on the assignment, try
to restrict the discussion to EdD as much as possible, so that there is no doubt as to the extent of your
collaboration.